# The Associated Press and Life Expectancy

**Story:** [AP analysis: Unemployment, income affect life expectancy](https://www.apnews.com/66ac44186b6249709501f07a7eab36da)

**Author:** Nicky Forster, Associated Press

**Topics:** Census Data, Linear Regression

**Datasets**

* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - **R12221544.txt** is the data dictionary
* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - Table B06009: Educational Attainment
    - Table B03002: Race
    - Table B19013: Median income
    - Table C17002: Ratio of income to poverty level
    - **R12221544.txt** is the data dictionary
* **US_A.CSV:** life expectancy by census tract, from [USALEEP](https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html)
    - **Record_Layout_CensusTract_Life_Expectancy.pdf** is data dictionary

# What's the story?

We're trying to figure out how the **life expectancy in a census tract** is related to other factors like unemployment, income, and others.

# PREPWORK BONUS!

Download the data yourself from Social Explorer and USALEEP (linked above) instead of relying on the data included.

## Reading in our data

### Read in `USA_A.CSV`

Rename any columns with weird or not-understandable names as something more descriptive.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statistics
from statistics import mode
from statistics import median
import re
import statsmodels.api as sm

In [2]:
df = pd.read_csv("US_A.csv")
df.head()

,Tract ID,STATE2KX,CNTY2KX,TRACT2KX,e(0),se(e(0)),Abridged life table flag
0,1001020100,1,1,20100,73.1,2.2348,3
1,1001020200,1,1,20200,76.9,3.3453,3
2,1001020400,1,1,20400,75.4,1.0216,3
3,1001020500,1,1,20500,79.4,1.1768,1
4,1001020600,1,1,20600,73.1,1.5519,3


In [3]:
df = df.rename(columns={'STATE2KX':'FIPS State Code','CNTY2KX':'FIPS Country Code', 
                        'TRACT2KX':'Census 2010 Tract', 
                        'Abridged life table flag':'Flag', 
                        'e(0)':'Life expectancy at birth',
                         'se(e(0))': 'Standard error'})
df.head() 

,Tract ID,FIPS State Code,FIPS Country Code,Census 2010 Tract,Life expectancy at birth,Standard error,Flag
0,1001020100,1,1,20100,73.1,2.2348,3
1,1001020200,1,1,20200,76.9,3.3453,3
2,1001020400,1,1,20400,75.4,1.0216,3
3,1001020500,1,1,20500,79.4,1.1768,1
4,1001020600,1,1,20600,73.1,1.5519,3


### Open `R12221544_SL140.csv`

You'll need to give an option to `pd.read_csv` to make sure it's read in successfully.

In [4]:
df1 = pd.read_csv("R12221544_SL140.csv", encoding = "ISO-8859-1")
df1.head()

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,ACS15_5yr_B23025005,ACS15_5yr_B23025006,ACS15_5yr_B23025007,ACS15_5yr_B23025001s,ACS15_5yr_B23025002s,ACS15_5yr_B23025003s,ACS15_5yr_B23025004s,ACS15_5yr_B23025005s,ACS15_5yr_B23025006s,ACS15_5yr_B23025007s
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama","Census Tract 201, Autauga County, Alabama",al,140,0,ACSSF,1760,NaN,...,54,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788
1,1001020200,14000US01001020200,"Census Tract 202, Autauga County, Alabama","Census Tract 202, Autauga County, Alabama",al,140,0,ACSSF,1761,NaN,...,116,15,847,143.030303,115.151515,114.545455,107.272727,38.181818,14.545455,86.666667
2,1001020300,14000US01001020300,"Census Tract 203, Autauga County, Alabama","Census Tract 203, Autauga County, Alabama",al,140,0,ACSSF,1762,NaN,...,91,8,990,169.090909,132.121212,134.545455,123.030303,31.515152,8.484848,120.606061
3,1001020400,14000US01001020400,"Census Tract 204, Autauga County, Alabama","Census Tract 204, Autauga County, Alabama",al,140,0,ACSSF,1763,NaN,...,216,15,1411,197.575758,157.575758,161.818182,132.121212,58.787879,14.545455,127.878788
4,1001020500,14000US01001020500,"Census Tract 205, Autauga County, Alabama","Census Tract 205, Autauga County, Alabama",al,140,0,ACSSF,1764,NaN,...,221,203,2737,321.818182,339.393939,356.969697,369.090909,89.090909,103.030303,273.939394


#### Filter out any columns we aren't interestd in

In [5]:
df1 = df1[['Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_STUSAB', 'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US',
      'Geo_REGION', 'Geo_DIVISION', 'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY','Geo_TRACT', 'ACS15_5yr_B23025001',
      'ACS15_5yr_B23025002', 'ACS15_5yr_B23025003', 'ACS15_5yr_B23025004',
      'ACS15_5yr_B23025005', 'ACS15_5yr_B23025006', 'ACS15_5yr_B23025007',
      'ACS15_5yr_B23025001s', 'ACS15_5yr_B23025002s', 'ACS15_5yr_B23025003s',
      'ACS15_5yr_B23025004s', 'ACS15_5yr_B23025005s', 'ACS15_5yr_B23025006s',
      'ACS15_5yr_B23025007s']]
df1 = df1.rename(columns={'Geo_TRACT':'TRACT', 'ACS15_5yr_B23025001':'Pop', 'ACS15_5yr_B23025002':'Pop_in_Labor_Force', 'ACS15_5yr_B23025003':'Pop_in_Civilian_Labor_Force', 'ACS15_5yr_B23025004':'Pop_in_Civilian_Labor_Force_employed',
      'ACS15_5yr_B23025005':'Pop_in_Civilian_Labor_Force_unemployed', 'ACS15_5yr_B23025006':'Pop_in_Labor_Force_Armed_Forces', 'ACS15_5yr_B23025007':'Pop_Not_in_Labor_Force',
      'ACS15_5yr_B23025001s':'Error_Pop', 'ACS15_5yr_B23025002s':'Error_Pop_in_Labor_Force', 'ACS15_5yr_B23025003s':'Error_Pop_in_Civilian_Labor_Force',
      'ACS15_5yr_B23025004s':'Error_Pop_in_Civilian_Labor_Force_Employed', 'ACS15_5yr_B23025005s':'Error_Pop_in_Civilian_Labor_Force_Unemployed', 'ACS15_5yr_B23025006s':'Error_Pop_in_Labor_Force_Armed_Forces',
      'ACS15_5yr_B23025007s':'Error_Pop_Not_in_Labor_Force'})
df1


,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_STUSAB,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,...,Pop_in_Civilian_Labor_Force_unemployed,Pop_in_Labor_Force_Armed_Forces,Pop_Not_in_Labor_Force,Error_Pop,Error_Pop_in_Labor_Force,Error_Pop_in_Civilian_Labor_Force,Error_Pop_in_Civilian_Labor_Force_Employed,Error_Pop_in_Civilian_Labor_Force_Unemployed,Error_Pop_in_Labor_Force_Armed_Forces,Error_Pop_Not_in_Labor_Force
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",al,0,ACSSF,1760,NaN,NaN,NaN,...,54,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788
1,1001020200,14000US01001020200,"Census Tract 202, Autauga County, Alabama",al,0,ACSSF,1761,NaN,NaN,NaN,...,116,15,847,143.030303,115.151515,114.545455,107.272727,38.181818,14.545455,86.666667
2,1001020300,14000US01001020300,"Census Tract 203, Autauga County, Alabama",al,0,ACSSF,1762,NaN,NaN,NaN,...,91,8,990,169.090909,132.121212,134.545455,123.030303,31.515152,8.484848,120.606061
3,1001020400,14000US01001020400,"Census Tract 204, Autauga County, Alabama",al,0,ACSSF,1763,NaN,NaN,NaN,...,216,15,1411,197.575758,157.575758,161.818182,132.121212,58.787879,14.545455,127.878788
4,1001020500,14000US01001020500,"Census Tract 205, Autauga County, Alabama",al,0,ACSSF,1764,NaN,NaN,NaN,...,221,203,2737,321.818182,339.393939,356.969697,369.090909,89.090909,103.030303,273.939394
5,1001020600,14000US01001020600,"Census Tract 206, Autauga County, Alabama",al,0,ACSSF,1765,NaN,NaN,NaN,...,190,52,1053,206.060606,160.606061,158.787879,139.393939,63.030303,30.303030,151.515152
6,1001020700,14000US01001020700,"Census Tract 207, Autauga County, Alabama",al,0,ACSSF,1766,NaN,NaN,NaN,...,150,0,796,163.636364,130.303030,130.303030,125.454545,47.878788,6.666667,80.606061
7,1001020801,14000US01001020801,"Census Tract 208.01, Autauga County, Alabama",al,0,ACSSF,1767,NaN,NaN,NaN,...,135,28,854,127.272727,124.242424,122.424242,136.969697,47.878788,13.939394,117.575758
8,1001020802,14000US01001020802,"Census Tract 208.02, Autauga County, Alabama",al,0,ACSSF,1768,NaN,NaN,NaN,...,415,124,3201,379.393939,269.696970,276.363636,250.303030,112.121212,47.878788,265.454545
9,1001020900,14000US01001020900,"Census Tract 209, Autauga County, Alabama",al,0,ACSSF,1769,NaN,NaN,NaN,...,194,0,1799,303.636364,218.787879,218.787879,214.545455,91.515152,9.696970,181.818182


#### Create a new column for percent unemployment

We'll be using the total population in the census tract as the baseline for employment.

In [6]:
df1['unemployment_percentage'] = (df1['Pop_in_Civilian_Labor_Force_unemployed']*100)/df1['Pop']
df1.head()

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_STUSAB,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,...,Pop_in_Labor_Force_Armed_Forces,Pop_Not_in_Labor_Force,Error_Pop,Error_Pop_in_Labor_Force,Error_Pop_in_Civilian_Labor_Force,Error_Pop_in_Civilian_Labor_Force_Employed,Error_Pop_in_Civilian_Labor_Force_Unemployed,Error_Pop_in_Labor_Force_Armed_Forces,Error_Pop_Not_in_Labor_Force,unemployment_percentage
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",al,0,ACSSF,1760,NaN,NaN,NaN,...,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788,3.474903
1,1001020200,14000US01001020200,"Census Tract 202, Autauga County, Alabama",al,0,ACSSF,1761,NaN,NaN,NaN,...,15,847,143.030303,115.151515,114.545455,107.272727,38.181818,14.545455,86.666667,6.701329
2,1001020300,14000US01001020300,"Census Tract 203, Autauga County, Alabama",al,0,ACSSF,1762,NaN,NaN,NaN,...,8,990,169.090909,132.121212,134.545455,123.030303,31.515152,8.484848,120.606061,3.696182
3,1001020400,14000US01001020400,"Census Tract 204, Autauga County, Alabama",al,0,ACSSF,1763,NaN,NaN,NaN,...,15,1411,197.575758,157.575758,161.818182,132.121212,58.787879,14.545455,127.878788,6.308411
4,1001020500,14000US01001020500,"Census Tract 205, Autauga County, Alabama",al,0,ACSSF,1764,NaN,NaN,NaN,...,203,2737,321.818182,339.393939,356.969697,369.090909,89.090909,103.030303,273.939394,2.695779


## Merging the data

Merge the dataframes together based on their census tract.

In [7]:
merged = pd.merge(df, df1, left_on='Census 2010 Tract', right_on='TRACT')
merged.head()

,Tract ID,FIPS State Code,FIPS Country Code,Census 2010 Tract,Life expectancy at birth,Standard error,Flag,Geo_FIPS,Geo_GEOID,Geo_NAME,...,Pop_in_Labor_Force_Armed_Forces,Pop_Not_in_Labor_Force,Error_Pop,Error_Pop_in_Labor_Force,Error_Pop_in_Civilian_Labor_Force,Error_Pop_in_Civilian_Labor_Force_Employed,Error_Pop_in_Civilian_Labor_Force_Unemployed,Error_Pop_in_Labor_Force_Armed_Forces,Error_Pop_Not_in_Labor_Force,unemployment_percentage
0,1001020100,1,1,20100,73.1,2.2348,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788,3.474903
1,1001020100,1,1,20100,73.1,2.2348,3,1033020100,14000US01033020100,"Census Tract 201, Colbert County, Alabama",...,0,1505,135.151515,115.151515,115.151515,108.484848,53.939394,6.666667,124.848485,4.676985
2,1001020100,1,1,20100,73.1,2.2348,3,1045020100,14000US01045020100,"Census Tract 201, Dale County, Alabama",...,0,1359,156.363636,143.030303,143.030303,132.727273,40.606061,6.666667,115.757576,4.012899
3,1001020100,1,1,20100,73.1,2.2348,3,1057020100,14000US01057020100,"Census Tract 201, Fayette County, Alabama",...,0,1228,179.393939,139.393939,139.393939,143.030303,30.303030,6.666667,118.787879,3.613995
4,1001020100,1,1,20100,73.1,2.2348,3,1127020100,14000US01127020100,"Census Tract 201, Walker County, Alabama",...,0,1271,179.393939,122.424242,122.424242,128.484848,69.090909,6.666667,130.303030,7.798507


In [8]:
# merged = merged.dropna()

In [9]:
merged.shape

(2729847, 36)

## Running the regression

Using the `statsmodels` package, run a linear regression to find the coefficient relating unemployment and life expectancy.

In [10]:
X = sm.add_constant(merged[['unemployment_percentage']])
mod = sm.OLS(merged['Life expectancy at birth'], X, missing='drop')
res = mod.fit()
res.summary()

/Users/lindseyfeingold/.pyenv/versions/3.7.1/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     Life expectancy at birth   R-squared:                       0.001
Model:                                  OLS   Adj. R-squared:                  0.001
Method:                       Least Squares   F-statistic:                     2293.
Date:                      Tue, 16 Jul 2019   Prob (F-statistic):               0.00
Time:                              16:56:14   Log-Likelihood:            -7.7548e+06
No. Observations:                   2728570   AIC:                         1.551e+07
Df Residuals:                       2728568   BIC:                         1.551e+07
Df Model:                                 1                                         
Covariance Type:                  nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      77.2391      0.005   1.59e+04      0.000      77.230      77.249
unemployment_percentage    -0.0365      0.001    -47.884      0.000      -0.038      -0.035
==============================================================================
Omnibus:                     8361.520   Durbin-Watson:                   0.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9710.959
Skew:                          -0.082   Prob(JB):                         0.00
Kurtosis:                       3.242   Cond. No.                         12.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Translate that into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy"**

In [11]:
# every 1 percentage point change in unemployment translates to a -0.0365 change in life expectancy


## Bringing more columns into the mix

Only dealing with unemployment seems kind of narrow-minded, let's expand our reach a bit.

### Read in `R12221544_SL140.csv`

It's also from the Census, and has many, many, many more columns available to you compared to the list dataset.

In [12]:
df4 = pd.read_csv("R12221550_SL140.csv", encoding = "ISO-8859-1")
df4.head()

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,ACS15_5yr_B23025005,ACS15_5yr_B23025006,ACS15_5yr_B23025007,ACS15_5yr_B23025001s,ACS15_5yr_B23025002s,ACS15_5yr_B23025003s,ACS15_5yr_B23025004s,ACS15_5yr_B23025005s,ACS15_5yr_B23025006s,ACS15_5yr_B23025007s
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama","Census Tract 201, Autauga County, Alabama",al,140,0,ACSSF,1760,NaN,...,54,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788
1,1001020200,14000US01001020200,"Census Tract 202, Autauga County, Alabama","Census Tract 202, Autauga County, Alabama",al,140,0,ACSSF,1761,NaN,...,116,15,847,143.030303,115.151515,114.545455,107.272727,38.181818,14.545455,86.666667
2,1001020300,14000US01001020300,"Census Tract 203, Autauga County, Alabama","Census Tract 203, Autauga County, Alabama",al,140,0,ACSSF,1762,NaN,...,91,8,990,169.090909,132.121212,134.545455,123.030303,31.515152,8.484848,120.606061
3,1001020400,14000US01001020400,"Census Tract 204, Autauga County, Alabama","Census Tract 204, Autauga County, Alabama",al,140,0,ACSSF,1763,NaN,...,216,15,1411,197.575758,157.575758,161.818182,132.121212,58.787879,14.545455,127.878788
4,1001020500,14000US01001020500,"Census Tract 205, Autauga County, Alabama","Census Tract 205, Autauga County, Alabama",al,140,0,ACSSF,1764,NaN,...,221,203,2737,321.818182,339.393939,356.969697,369.090909,89.090909,103.030303,273.939394


Using this census data, create a new dataframe that includes the following columns:

* Percent unemployed
* Percents Black, White, and Hispanic
* Median Income (in increments of 10,000 dollars)
* Percent of the population with less than a high school education
* Percent of the population between 1-1.5x the poverty line

If you have to many any editorial decisions about which columns you choose or how you do your math, please explain them.

In [13]:
df4 = pd.read_csv("R12221550_SL140.csv", encoding = "ISO-8859-1")

df4 = df4[['Geo_TRACT', 'ACS15_5yr_B03002001', 'ACS15_5yr_B03002003', 'ACS15_5yr_B03002004', 'ACS15_5yr_B03002012', 'ACS15_5yr_B19013001', 'ACS15_5yr_B06009001','ACS15_5yr_B06009002', 'ACS15_5yr_C17002001', 'ACS15_5yr_C17002004', 'ACS15_5yr_C17002005', 'ACS15_5yr_B23025005','ACS15_5yr_B23025001', 'Geo_FIPS']] 

df4 = df4.rename(columns={'Geo_TRACT':'TRACT', 'ACS15_5yr_B03002001':'total_pop', 'ACS15_5yr_B03002003':'white',
      'ACS15_5yr_B03002004':'black', 'ACS15_5yr_B03002012':'hispanic', 'ACS15_5yr_B19013001':'median_income', 'ACS15_5yr_B06009001':'total_high_school','ACS15_5yr_B06009002':'less_high_school', 'ACS15_5yr_C17002001':'total_poverty', 'ACS15_5yr_C17002004':'pop_between_1_to_1.24',
      'ACS15_5yr_C17002005':'pop_between_1.25_to_1.49', 'ACS15_5yr_B23025005' : 'unemployment', 'ACS15_5yr_B23025001': 'total_pop_employ'})

df4['percent_unemployed'] = (df4['unemployment']*100)/df4['total_pop_employ']

df4['white'] = (df4['white'] / df4['total_pop']) * 100
df4['black'] = (df4['black'] / df4['total_pop']) * 100
df4['hispanic'] = (df4['hispanic'] / df4['total_pop']) * 100

df4['less_high_school']= df4['less_high_school']/df4['total_pop']*100

df4['percent_poverty_pop'] = (df4['pop_between_1_to_1.24'] + df4['pop_between_1.25_to_1.49']) / df4['total_pop']*100

df4['median_income'] = df4['median_income']/10000


df4.head()   


,TRACT,total_pop,white,black,hispanic,median_income,total_high_school,less_high_school,total_poverty,pop_between_1_to_1.24,pop_between_1.25_to_1.49,unemployment,total_pop_employ,Geo_FIPS,percent_unemployed,percent_poverty_pop
0,20100,1948,87.422998,7.700205,0.872690,6.1838,1243.0,9.445585,1948,81,101,54,1554,1001020100,3.474903,9.342916
1,20200,2156,40.445269,53.293135,0.788497,3.2303,1397.0,16.512059,1983,232,58,116,1731,1001020200,6.701329,13.450835
2,20300,2968,74.528302,18.564690,0.000000,4.4922,2074.0,7.446092,2968,148,207,91,2462,1001020300,3.696182,11.960916
3,20400,4423,82.794483,3.662672,10.490617,5.4329,2899.0,7.664481,4423,141,182,216,3424,1001020400,6.308411,7.302736
4,20500,10763,68.456750,24.844374,0.743287,5.1965,6974.0,2.880238,10563,256,1064,221,8198,1001020500,2.695779,12.264239


### Join your datasets

Combine your life expectancy dataset with this census dataset to create a new dataframe.

In [14]:
merged1 = pd.merge(merged, df4, on='Geo_FIPS')
merged1.head()

,Tract ID,FIPS State Code,FIPS Country Code,Census 2010 Tract,Life expectancy at birth,Standard error,Flag,Geo_FIPS,Geo_GEOID,Geo_NAME,...,median_income,total_high_school,less_high_school,total_poverty,pop_between_1_to_1.24,pop_between_1.25_to_1.49,unemployment,total_pop_employ,percent_unemployed,percent_poverty_pop
0,1001020100,1,1,20100,73.1,2.2348,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,6.1838,1243.0,9.445585,1948,81,101,54,1554,3.474903,9.342916
1,1033020100,1,33,20100,72.3,1.4305,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,6.1838,1243.0,9.445585,1948,81,101,54,1554,3.474903,9.342916
2,1045020100,1,45,20100,75.0,1.5468,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,6.1838,1243.0,9.445585,1948,81,101,54,1554,3.474903,9.342916
3,1057020100,1,57,20100,72.6,1.4231,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,6.1838,1243.0,9.445585,1948,81,101,54,1554,3.474903,9.342916
4,1127020100,1,127,20100,68.6,1.3919,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,6.1838,1243.0,9.445585,1948,81,101,54,1554,3.474903,9.342916


## Running your multivariate regression

Using the `statsmodels` package and this new dataframe, run a multivariate linear regression to find the coefficient relating your columns and life expectancy.

In [15]:
X = sm.add_constant(merged1[['percent_unemployed', 'white','black','hispanic','less_high_school','percent_poverty_pop', 'median_income']])
y = merged1['Life expectancy at birth']
mod = sm.OLS(y, X, missing='drop')
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     Life expectancy at birth   R-squared:                       0.004
Model:                                  OLS   Adj. R-squared:                  0.004
Method:                       Least Squares   F-statistic:                     1376.
Date:                      Tue, 16 Jul 2019   Prob (F-statistic):               0.00
Time:                              17:20:28   Log-Likelihood:            -7.6933e+06
No. Observations:                   2708347   AIC:                         1.539e+07
Df Residuals:                       2708339   BIC:                         1.539e+07
Df Model:                                 7                                         
Covariance Type:                  nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  76.3984      0.034   2267.391      0.000      76.332      76.464
percent_unemployed     -0.0090      0.001     -9.388      0.000      -0.011      -0.007
white                   0.0007      0.000      2.216      0.027    7.81e-05       0.001
black                3.951e-05      0.000      0.123      0.902      -0.001       0.001
hispanic             4.885e-05      0.000      0.141      0.888      -0.001       0.001
less_high_school        0.0115      0.001     20.296      0.000       0.010       0.013
percent_poverty_pop    -0.0032      0.001     -5.503      0.000      -0.004      -0.002
median_income           0.1224      0.002     72.390      0.000       0.119       0.126
==============================================================================
Omnibus:                     8834.225   Durbin-Watson:                   1.717
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10331.903
Skew:                          -0.084   Prob(JB):                         0.00
Kurtosis:                       3.252   Cond. No.                     1.02e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Translate some of your coefficients into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy."** Do this with numbers that are meaningful, and in a way that is easily understandable to your reader.

In [ ]:
#every 1 percentage point change in unemployment translates to a -0.0090 change in life expectancy.